In [5]:
import tensorflow as tf
import numpy as np
import pandas as pd
import json
import nltk
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Input, Embedding, LSTM , Dense,GlobalMaxPooling1D,Flatten
from tensorflow.keras.models import Model
import matplotlib.pyplot as plt

In [6]:
with open('../data/chat/content.json') as content:
    data1 = json.load(content)

#getting all the data to lists
tags = []
inputs = []
responses={}
for intent in data1['intents']:
    responses[intent['tag']]=intent['responses']
    for lines in intent['input']:
        inputs.append(lines)
        tags.append(intent['tag'])
#converting to dataframe


In [10]:
data = pd.DataFrame({"inputs":inputs, "tags":tags})
print(data)

                                          inputs           tags
0                                          hello       greeting
1                                       hi there       greeting
2                               nice to meet you       greeting
3       hi, is this is the pirate's organization       greeting
4                             any pirates here ?       greeting
..                                           ...            ...
76         tell me more about the pre requisites  prerequisites
77    give more details about the pre-requisites  prerequisites
78                     What are the requirements  prerequisites
79                 what's the basic requirements  prerequisites
80  what are the basic skills to get recruited ?  prerequisites

[81 rows x 2 columns]


In [11]:
import string
data['inputs'] = data['inputs'].apply(lambda wrd:[ltrs.lower() for ltrs in wrd if ltrs not in string.punctuation])
data['inputs'] = data['inputs'].apply(lambda wrd: ''.join(wrd))
#tokenize the data
from tensorflow.keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer(num_words=2000)
tokenizer.fit_on_texts(data['inputs'])
train = tokenizer.texts_to_sequences(data['inputs'])

#apply padding
from tensorflow.keras.preprocessing.sequence import pad_sequences
x_train = pad_sequences(train)

#encoding the outputs
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y_train = le.fit_transform(data['tags'])

In [12]:
input_shape = x_train.shape[1]
print(input_shape)
#define vocabulary
vocabulary = len(tokenizer.word_index)
print("number of unique words : ",vocabulary)
#output length
output_length = le.classes_.shape[0]
print("output length: ",output_length)

8
number of unique words :  96
output length:  8


In [13]:
i = Input(shape=(input_shape,))
x = Embedding(vocabulary+1,10)(i)
x = LSTM(10,return_sequences=True)(x)
x = Flatten()(x)
x = Dense(output_length,activation="softmax")(x)
model  = Model(i,x)
#compiling the model
model.compile(loss="sparse_categorical_crossentropy",optimizer='adam',metrics=['accuracy'])
#training the model
train = model.fit(x_train,y_train,epochs=200)

Epoch 1/200
3/3 [==============================] - 1s 0s/step - loss: 2.0807 - accuracy: 0.0741
Epoch 2/200
3/3 [==============================] - 0s 0s/step - loss: 2.0771 - accuracy: 0.1481
Epoch 3/200
3/3 [==============================] - 0s 0s/step - loss: 2.0737 - accuracy: 0.1852
Epoch 4/200
3/3 [==============================] - 0s 0s/step - loss: 2.0703 - accuracy: 0.2222
Epoch 5/200
3/3 [==============================] - 0s 8ms/step - loss: 2.0666 - accuracy: 0.2222
Epoch 6/200
3/3 [==============================] - 0s 8ms/step - loss: 2.0630 - accuracy: 0.2222
Epoch 7/200
3/3 [==============================] - 0s 8ms/step - loss: 2.0589 - accuracy: 0.2099
Epoch 8/200
3/3 [==============================] - 0s 8ms/step - loss: 2.0543 - accuracy: 0.2099
Epoch 9/200
3/3 [==============================] - 0s 0s/step - loss: 2.0499 - accuracy: 0.2099
Epoch 10/200
3/3 [==============================] - 0s 0s/step - loss: 2.0445 - accuracy: 0.2099
Epoch 11/200
3/3 [=================

In [14]:
import random
while True:
    texts_p = []
    prediction_input = input('You : ')
    #removing punctuation and converting to lowercase
    prediction_input = [letters.lower() for letters in prediction_input if letters not in string.punctuation]
    prediction_input = ''.join(prediction_input)
    texts_p.append(prediction_input)
    #tokenizing and padding
    prediction_input = tokenizer.texts_to_sequences(texts_p)
    prediction_input = np.array(prediction_input).reshape(-1)
    prediction_input = pad_sequences([prediction_input],input_shape)
    #getting output from model
    output = model.predict(prediction_input)
    output = output.argmax()
    #finding the right tag and predicting
    response_tag = le.inverse_transform([output])[0]
    print("Going Merry : ",random.choice(responses[response_tag]))
    if response_tag == "goodbye":
        break

Going Merry :  Hey yo, do you need help ?
Going Merry :  Ha , Glad you showed up. How are you ?
Going Merry :  Ha , Glad you showed up. How are you ?
Going Merry :  My name is Going Merry and I am here to answer your questions regarding our organization. go ahead and shoot
Going Merry :  Hi random person, what do'ya want ?
Going Merry :  welcome aboard, how may I help you ?
Going Merry :  nice conversation. bye
